
DATA 603 – Big Data Platforms

Homework #11 ML + AWS

Write a Spark ML-lib program to predict flight delays for certain airline UMAIR. UMAIR operates throughout the US. 

In [1]:
from pyspark import SparkContext 
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.util import MLUtils

from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.linalg import Vectors

In [2]:
#generate random data for flights of size about 100 rows.

import random
from datetime import date, time
import csv

list_airport_code=['BWI', 'CAE', 'EEN', 'PHL', 'LAX', 'JFK', 'MIA']

# random Day generator
def generate_date():

    start_dt = date.today().replace(day=1, month=1, year=2019).toordinal()
    end_dt = date.today().toordinal()
    random_day = date.fromordinal(random.randint(start_dt, end_dt))
    return random_day

# random Time generator
def generate_time():
    hours = random.randint(3,23)
    minutes= random.randint(0,59)
    
    t1 = time(hours-(random.randint(0,2)), minutes) #flight schedule land time
    t2 = time(hours, minutes) # flight actual land time
    
    format = "%H:%M"
    #format datetime using strftime()
    start_time = t1.strftime(format)
    end_time= t2.strftime(format)
    
    return [start_time, end_time]

 
#Final function
def createFakeData(name_of_file, number_of_rows):
    with open (name_of_file, 'w', newline='') as file:
        writeData = csv.writer(file)

        #The headers
        writeData.writerow(['Flight No', 
                            'Date', 
                            'Time',
                            'Actual Date',
                            'Actual Time', 
                            'Airports'])
        
        for i in range(1,number_of_rows):
            
            day=generate_date()
            time=generate_time()
            aiport_codes = random.sample(list_airport_code, 2)
            
            writeData.writerow(['UM-'+ str(random.randint(400,799)+i),
                                 day, 
                                 time[0] ,
                                 day,
                                 time[1],
                                 aiport_codes[0]+ '-' + aiport_codes[1]
                               ])
                                
                                
    #print('You created a CSV file: ' + str(name_of_file) + ' with ' + str(number_of_rows) + ' rows.' )
            
createFakeData('FlightData.csv', 101)                                 

In [3]:
#https://medium.com/kharpann/perform-linear-regression-on-big-data-using-python-spark-and-mllib-b1204769547e

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('flights delay').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [6]:
# Load training data
df1 = spark.read.csv("FlightData.csv", header='true', inferSchema='false')
#spark.read.format("csv").load("FlightData.csv")
df1.show(5)

+---------+----------+-----+-----------+-----------+--------+
|Flight No|      Date| Time|Actual Date|Actual Time|Airports|
+---------+----------+-----+-----------+-----------+--------+
|   UM-754|2019-02-08|02:01| 2019-02-08|      04:01| CAE-JFK|
|   UM-784|2019-04-17|13:32| 2019-04-17|      15:32| EEN-JFK|
|   UM-778|2019-06-05|03:48| 2019-06-05|      04:48| JFK-LAX|
|   UM-616|2019-04-03|07:31| 2019-04-03|      07:31| MIA-LAX|
|   UM-793|2019-02-10|14:12| 2019-02-10|      14:12| MIA-JFK|
+---------+----------+-----+-----------+-----------+--------+
only showing top 5 rows



In [7]:
df1.printSchema()

root
 |-- Flight No: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Actual Date: string (nullable = true)
 |-- Actual Time: string (nullable = true)
 |-- Airports: string (nullable = true)



In [8]:
# Sort the dataframe in descending- sort by a single colunm

df2 = df1.orderBy('Date', ascending=False)
df2.show(5)

+---------+----------+-----+-----------+-----------+--------+
|Flight No|      Date| Time|Actual Date|Actual Time|Airports|
+---------+----------+-----+-----------+-----------+--------+
|   UM-743|2020-07-10|09:58| 2020-07-10|      11:58| MIA-LAX|
|   UM-526|2020-06-21|14:21| 2020-06-21|      14:21| PHL-JFK|
|   UM-641|2020-06-17|18:11| 2020-06-17|      19:11| JFK-BWI|
|   UM-831|2020-06-16|19:50| 2020-06-16|      20:50| LAX-BWI|
|   UM-883|2020-06-13|14:06| 2020-06-13|      14:06| CAE-BWI|
+---------+----------+-----+-----------+-----------+--------+
only showing top 5 rows



In [9]:

#Change columns type
for d in df2.columns:
    if d in ["Date", "Actual Date"]:
    # add condition for the cols to be type cast
       df2=df2.withColumn(d, df2[d].cast(DateType()))

In [10]:
df2.printSchema()

root
 |-- Flight No: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- Actual Date: date (nullable = true)
 |-- Actual Time: string (nullable = true)
 |-- Airports: string (nullable = true)



In [11]:
#df2.show(5)

In [12]:
#df2.printSchema()

In [13]:
#df2.describe().show()

In [14]:
# define timedelta function (obtain delay duration in minutes)
from pyspark.sql.functions import udf

def time_delta(y,x): 
    from datetime import datetime
    end = datetime.strptime(y, '%H:%M')
    start = datetime.strptime(x, '%H:%M')
    delta = (end-start).seconds
    return delta

# register as a UDF 
f = udf(time_delta, IntegerType())

# Apply function
df_new = df2.withColumn('Delay Duration', f(df2["Actual Time"], df2["Time"])/60)

In [15]:
df_new.show(5)

+---------+----------+-----+-----------+-----------+--------+--------------+
|Flight No|      Date| Time|Actual Date|Actual Time|Airports|Delay Duration|
+---------+----------+-----+-----------+-----------+--------+--------------+
|   UM-743|2020-07-10|09:58| 2020-07-10|      11:58| MIA-LAX|         120.0|
|   UM-526|2020-06-21|14:21| 2020-06-21|      14:21| PHL-JFK|           0.0|
|   UM-641|2020-06-17|18:11| 2020-06-17|      19:11| JFK-BWI|          60.0|
|   UM-831|2020-06-16|19:50| 2020-06-16|      20:50| LAX-BWI|          60.0|
|   UM-883|2020-06-13|14:06| 2020-06-13|      14:06| CAE-BWI|           0.0|
+---------+----------+-----+-----------+-----------+--------+--------------+
only showing top 5 rows



In [16]:
df_new.describe().show()

+-------+---------+-----+-----------+--------+-----------------+
|summary|Flight No| Time|Actual Time|Airports|   Delay Duration|
+-------+---------+-----+-----------+--------+-----------------+
|  count|      100|  100|        100|     100|              100|
|   mean|     null| null|       null|    null|             51.6|
| stddev|     null| null|       null|    null|50.46691085165696|
|    min|   UM-426|01:57|      03:17| BWI-CAE|              0.0|
|    max|   UM-883|23:25|      23:25| PHL-MIA|            120.0|
+-------+---------+-----+-----------+--------+-----------------+



In [17]:
df_new.printSchema()

root
 |-- Flight No: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- Actual Date: date (nullable = true)
 |-- Actual Time: string (nullable = true)
 |-- Airports: string (nullable = true)
 |-- Delay Duration: double (nullable = true)



### Use OneHotEncoder to encode my categorical data.

In [18]:
 # Category Indexing with StringIndexer

stringIndexer = StringIndexer(inputCol = "Airports", outputCol = "Airports"+ 'Index')
df_new= stringIndexer.fit(df_new).transform(df_new)
#indexer.show(5) 

# stringIndexer = StringIndexer(inputCol = "Destination", outputCol = "Destination"+ 'Index')
# df_new= stringIndexer.fit(df_new).transform(df_new)
df_new.show(5) 


+---------+----------+-----+-----------+-----------+--------+--------------+-------------+
|Flight No|      Date| Time|Actual Date|Actual Time|Airports|Delay Duration|AirportsIndex|
+---------+----------+-----+-----------+-----------+--------+--------------+-------------+
|   UM-743|2020-07-10|09:58| 2020-07-10|      11:58| MIA-LAX|         120.0|          6.0|
|   UM-526|2020-06-21|14:21| 2020-06-21|      14:21| PHL-JFK|           0.0|         19.0|
|   UM-641|2020-06-17|18:11| 2020-06-17|      19:11| JFK-BWI|          60.0|         21.0|
|   UM-831|2020-06-16|19:50| 2020-06-16|      20:50| LAX-BWI|          60.0|         23.0|
|   UM-883|2020-06-13|14:06| 2020-06-13|      14:06| CAE-BWI|           0.0|         20.0|
+---------+----------+-----+-----------+-----------+--------+--------------+-------------+
only showing top 5 rows



In [19]:
# Convert the dependent variable(Delay Duration) into dependent variable indices using the StringIndexer
dependent_stringIndexer = StringIndexer(inputCol = "Delay Duration", outputCol = "Delay Duration"+ 'Index')
df_new = dependent_stringIndexer.fit(df_new).transform(df_new)
df_new.show(5)

+---------+----------+-----+-----------+-----------+--------+--------------+-------------+-------------------+
|Flight No|      Date| Time|Actual Date|Actual Time|Airports|Delay Duration|AirportsIndex|Delay DurationIndex|
+---------+----------+-----+-----------+-----------+--------+--------------+-------------+-------------------+
|   UM-743|2020-07-10|09:58| 2020-07-10|      11:58| MIA-LAX|         120.0|          6.0|                1.0|
|   UM-526|2020-06-21|14:21| 2020-06-21|      14:21| PHL-JFK|           0.0|         19.0|                0.0|
|   UM-641|2020-06-17|18:11| 2020-06-17|      19:11| JFK-BWI|          60.0|         21.0|                2.0|
|   UM-831|2020-06-16|19:50| 2020-06-16|      20:50| LAX-BWI|          60.0|         23.0|                2.0|
|   UM-883|2020-06-13|14:06| 2020-06-13|      14:06| CAE-BWI|           0.0|         20.0|                0.0|
+---------+----------+-----+-----------+-----------+--------+--------------+-------------+-------------------+
o

In [20]:
# # Use OneHotEncoder to convert categorical variables into binary SparseVectors

# encoder = OneHotEncoderEstimator(inputCols=["DestinationIndex"], outputCols=["DestinationVec"], dropLast=False)


In [21]:
# model = encoder.fit(df_new)
# df_new= model.transform(df_new)
# #df_new.show(5)

In [22]:
encoder = OneHotEncoderEstimator(inputCols=["AirportsIndex"], outputCols=["AirportsVec"])#, dropLast=False)

In [23]:
model1 = encoder.fit(df_new)
df_new= model1.transform(df_new)
df_new.show(5)

+---------+----------+-----+-----------+-----------+--------+--------------+-------------+-------------------+---------------+
|Flight No|      Date| Time|Actual Date|Actual Time|Airports|Delay Duration|AirportsIndex|Delay DurationIndex|    AirportsVec|
+---------+----------+-----+-----------+-----------+--------+--------------+-------------+-------------------+---------------+
|   UM-743|2020-07-10|09:58| 2020-07-10|      11:58| MIA-LAX|         120.0|          6.0|                1.0| (39,[6],[1.0])|
|   UM-526|2020-06-21|14:21| 2020-06-21|      14:21| PHL-JFK|           0.0|         19.0|                0.0|(39,[19],[1.0])|
|   UM-641|2020-06-17|18:11| 2020-06-17|      19:11| JFK-BWI|          60.0|         21.0|                2.0|(39,[21],[1.0])|
|   UM-831|2020-06-16|19:50| 2020-06-16|      20:50| LAX-BWI|          60.0|         23.0|                2.0|(39,[23],[1.0])|
|   UM-883|2020-06-13|14:06| 2020-06-13|      14:06| CAE-BWI|           0.0|         20.0|                0.0|(

In [24]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [25]:
# Transform all features into a vector using VectorAssembler

featureassembler = VectorAssembler().setInputCols([ "AirportsVec"]).setOutputCol("Independent Features")
output1 = featureassembler.transform(df_new)
output1.show(5)

+---------+----------+-----+-----------+-----------+--------+--------------+-------------+-------------------+---------------+--------------------+
|Flight No|      Date| Time|Actual Date|Actual Time|Airports|Delay Duration|AirportsIndex|Delay DurationIndex|    AirportsVec|Independent Features|
+---------+----------+-----+-----------+-----------+--------+--------------+-------------+-------------------+---------------+--------------------+
|   UM-743|2020-07-10|09:58| 2020-07-10|      11:58| MIA-LAX|         120.0|          6.0|                1.0| (39,[6],[1.0])|      (39,[6],[1.0])|
|   UM-526|2020-06-21|14:21| 2020-06-21|      14:21| PHL-JFK|           0.0|         19.0|                0.0|(39,[19],[1.0])|     (39,[19],[1.0])|
|   UM-641|2020-06-17|18:11| 2020-06-17|      19:11| JFK-BWI|          60.0|         21.0|                2.0|(39,[21],[1.0])|     (39,[21],[1.0])|
|   UM-831|2020-06-16|19:50| 2020-06-16|      20:50| LAX-BWI|          60.0|         23.0|                2.0|(3

In [26]:
 finalize_data = output1.select("Independent Features","Delay Duration")

In [27]:
finalize_data.show()

+--------------------+--------------+
|Independent Features|Delay Duration|
+--------------------+--------------+
|      (39,[6],[1.0])|         120.0|
|     (39,[19],[1.0])|           0.0|
|     (39,[21],[1.0])|          60.0|
|     (39,[23],[1.0])|          60.0|
|     (39,[20],[1.0])|           0.0|
|      (39,[9],[1.0])|         120.0|
|      (39,[7],[1.0])|          60.0|
|      (39,[3],[1.0])|          60.0|
|      (39,[5],[1.0])|         120.0|
|     (39,[22],[1.0])|           0.0|
|     (39,[31],[1.0])|           0.0|
|      (39,[5],[1.0])|          60.0|
|      (39,[0],[1.0])|         120.0|
|     (39,[18],[1.0])|           0.0|
|     (39,[12],[1.0])|           0.0|
|      (39,[1],[1.0])|         120.0|
|     (39,[10],[1.0])|          60.0|
|     (39,[14],[1.0])|         120.0|
|     (39,[11],[1.0])|           0.0|
|     (39,[33],[1.0])|         120.0|
+--------------------+--------------+
only showing top 20 rows



In [28]:
train_data, test_data = finalize_data.randomSplit([0.9, 0.1], seed = 100)
print("Training Dataset Count: " + str(train_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 87
Test Dataset Count: 13


In [29]:
#reate an instance of our model using the given columns and also instantiate a prediction named column for our predictions
lin_reg=LinearRegression(featuresCol='Independent Features',labelCol='Delay Duration',predictionCol='prediction')
lin_reg = lin_reg.fit(train_data)

In [30]:
lin_reg.coefficients

DenseVector([40.0, 45.0, -0.0, 60.0, 45.0, 90.0, 75.0, 30.0, -0.0, 80.0, 60.0, 60.0, 40.0, 80.0, 60.0, 60.0, 40.0, 60.0, 30.0, 30.0, 60.0, 60.0, 60.0, 60.0, -0.0, 120.0, 60.0, 60.0, 60.0, -0.0, 60.0, -0.0, -0.0, 120.0, 60.0, -0.0, 60.0, 120.0, -0.0])

In [31]:
lin_reg.intercept

1.658194722302727e-13

In [32]:
pred_results = lin_reg.evaluate(test_data)

In [33]:
pred_results.predictions.show()

+--------------------+--------------+--------------------+
|Independent Features|Delay Duration|          prediction|
+--------------------+--------------+--------------------+
|     (39,[23],[1.0])|          60.0|  60.000000000000014|
|      (39,[7],[1.0])|          60.0|  30.000000000000004|
|      (39,[1],[1.0])|         120.0|   45.00000000000001|
|     (39,[14],[1.0])|         120.0|   60.00000000000001|
|     (39,[25],[1.0])|           0.0|  120.00000000000006|
|      (39,[3],[1.0])|          60.0|   60.00000000000002|
|      (39,[3],[1.0])|          60.0|   60.00000000000002|
|      (39,[5],[1.0])|         120.0|   90.00000000000001|
|      (39,[7],[1.0])|          60.0|  30.000000000000004|
|      (39,[5],[1.0])|         120.0|   90.00000000000001|
|     (39,[10],[1.0])|         120.0|   60.00000000000001|
|     (39,[17],[1.0])|           0.0|  60.000000000000014|
|      (39,[2],[1.0])|         120.0|-5.04870979341447...|
+--------------------+--------------+-------------------

In [34]:
#Summarize the model over the training set
trainingSummary = lin_reg.summary

In [37]:
#Some probability calculations

print(pred_results.meanAbsoluteError)
print(pred_results.rootMeanSquaredError)
print(pred_results.r2)

47.307692307692314
61.28433103795155
-1.0035511363636371


In [36]:
#Vizualization

import matplotlib.pyplot as plt
import numpy as np
beta = np.sort(lin_reg.coefficients)
plt.plot(beta)
plt.ylabel('Beta Coefficients')
plt.show()

<Figure size 640x480 with 1 Axes>